In [10]:
import pandas as pd
import numpy as np
df = pd.read_csv('./datasets/DfCleaned.csv')

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import LSTM, Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

### Model n-gram2

In [12]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram2'].values)
x = tokenizer.texts_to_sequences(df['ngram2'].values)
x = pad_sequences(x)

In [13]:
y = pd.get_dummies(df['clarity']).values
#y = df['Class']

In [14]:
from sklearn.model_selection import KFold
# num_folds = 5
# kfold = KFold(n_splits=num_folds, shuffle=True)

In [15]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [16]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [17]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=6, batch_size=32, verbose=1)

In [18]:
result = cross_val_score(clf, x, y, scoring='accuracy', cv=2)
result

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2


2023-11-10 12:48:31.257023: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 53552000 exceeds 10% of free system memory.


125/125 [==============================] - 174s 1s/step - loss: 0.6323 - accuracy: 0.3785
Epoch 2/2
63/63 [==============================] - 16s 245ms/step


/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/2


2023-11-10 13:00:27.019879: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 53552000 exceeds 10% of free system memory.


125/125 [==============================] - 180s 1s/step - loss: 0.6391 - accuracy: 0.3450
Epoch 2/2
63/63 [==============================] - 16s 244ms/step


array([0.4775, 0.391 , 0.3805])

In [19]:
result.mean()

0.4163333333333334

### Model n-gram3

In [20]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df['ngram3'].values)
x = tokenizer.texts_to_sequences(df['ngram3'].values)
x = pad_sequences(x)

In [ ]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [21]:
embed_dim = 32
lstm_out = 32

model = Sequential([
    layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
    layers.Dense(1, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 5015, 32)          32000     
                                                                 
 conv1d_4 (Conv1D)           (None, 5015, 32)          3104      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 2507, 32)          0         
 g1D)                                                            
                                                                 
 lstm_4 (LSTM)               (None, 128)               82432     
                                                                 
 dense_8 (Dense)             (None, 1)                 129       
                                                                 
Total params: 117665 (459.63 KB)
Trainable params: 117665 (459.63 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [22]:
results = []

# Itera sobre os folds
for train_index, test_index in kfold.split(x):
    xtrain, xtest = x[train_index], x[test_index]
    ytrain, ytest = y[train_index], y[test_index]

    history = model.fit(xtrain, ytrain, epochs=3, batch_size=32, verbose=1, validation_data=(xtrain, ytrain), callbacks=[early_stopping])

    # Avalie o modelo no conjunto de teste para o fold atual
    loss, accuracy = model.evaluate(xtest, ytest, verbose=0, batch_size=32)
    results.append(accuracy)

# Calcula a média das métricas de avaliação
mean_accuracy = np.mean(results)
print("Mean cross-validated accuracy:", mean_accuracy)


NameError: name 'kfold' is not defined

### Model PoS

In [23]:
df_pos = pd.read_csv('./datasets/DfPos.csv')

In [24]:
max_features = 1000
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_pos['pos'].values)
x = tokenizer.texts_to_sequences(df_pos['pos'].values)
x = pad_sequences(x)

In [25]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [26]:
def modelLSTM():
    max_features = 1000
    embed_dim = 32
    lstm_out = 32
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= x.shape[1]),
        layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.LSTM(128, dropout=0.1, recurrent_dropout=0.1),
        layers.Dense(64, activation='relu'),
        layers.Dense(3, activation='softmax')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [27]:
clf = KerasClassifier(build_fn=modelLSTM, epochs=6, batch_size=32, verbose=1)

In [28]:
result = cross_val_score(clf, x, y, scoring='accuracy', cv=2)
result

/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/6


2023-11-10 13:10:43.637038: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43944000 exceeds 10% of free system memory.


52/94 [===============>..............] - ETA: 1:12 - loss: 0.6390 - accuracy: 0.3672

In [ ]:
result.mean()